In [ ]:
from dotenv import load_dotenv
from IPython.display import Pretty

from intelligence_layer.core import (
    DetectLanguage,
    DetectLanguageInput,
    InMemoryTracer,
    Language,
    LuminousControlModel,
    NoOpTracer,
)
from intelligence_layer.examples import (
    LongContextQa,
    LongContextQaInput,
    MultipleChunkQa,
    MultipleChunkQaInput,
    SingleChunkQa,
    SingleChunkQaInput,
)

load_dotenv()

# Question and Answer

A common use case for using large language models is to generate answers to questions based on a given piece of text.

<div class="alert alert-warning">

This notebook we will be focusing on the open-book Q&A use case, where we provide the model with a piece of text we think is relevant to the question and ask the model to answer the question based on the given text.
To make proper use of the classification task, it is necessary to evaluate the results in an iterative way, to ensure it satisfies your requirements.
For an example of how such an evaluation can look like, refer to [evaluation.ipynb](./evaluation.ipynb).
</div>


Let's grab a piece of text we want to ask a question about. We can start with a random Wikipedia article about ["Surface micromachining"](https://en.wikipedia.org/wiki/Surface_micromachining)

In [ ]:
text = """Surface micromachining

Surface micromachining builds microstructures by deposition and etching structural layers over a substrate.[1] This is different from Bulk micromachining, in which a silicon substrate wafer is selectively etched to produce structures.

Layers

Generally, polysilicon is used as one of the substrate layers while silicon dioxide is used as a sacrificial layer. The sacrificial layer is removed or etched out to create any necessary void in the thickness direction. Added layers tend to vary in size from 2-5 micrometres. The main advantage of this machining process is the ability to build electronic and mechanical components (functions) on the same substrate. Surface micro-machined components are smaller compared to their bulk micro-machined counterparts.

As the structures are built on top of the substrate and not inside it, the substrate's properties are not as important as in bulk micro-machining. Expensive silicon wafers can be replaced by cheaper substrates, such as glass or plastic. The size of the substrates may be larger than a silicon wafer, and surface micro-machining is used to produce thin-film transistors on large area glass substrates for flat panel displays. This technology can also be used for the manufacture of thin film solar cells, which can be deposited on glass, polyethylene terepthalate substrates or other non-rigid materials.

Fabrication process

Micro-machining starts with a silicon wafer or other substrate upon which new layers are grown. These layers are selectively etched by photo-lithography; either a wet etch involving an acid, or a dry etch involving an ionized gas (or plasma). Dry etching can combine chemical etching with physical etching or ion bombardment. Surface micro-machining involves as many layers as are needed with a different mask (producing a different pattern) on each layer. Modern integrated circuit fabrication uses this technique and can use as many as 100 layers. Micro-machining is a younger technology and usually uses no more than 5 or 6 layers. Surface micro-machining uses developed technology (although sometimes not enough for demanding applications) which is easily repeatable for volume production."""

We can use the `SingleChunkQa`-task to answer questions about this text!
This particular task is optimized for shorter texts that fit into the model's context window.
The main things of interest are that you can provide a `QaInput`, which consists of a `question` you want to ask and a `text` to base that answer upon.

The output will be a `QaOutput`, which will include an `answer` (if it can find one in the text) and `highlights` which mark the most relevant sections of the input text for the generated answer.


In [ ]:
# Define some question you want to ask about the input text
question = "What are some benefits of surface micro-machining?"

# Pass both the input text and the question to the SingleChunkQaInput-task
input = SingleChunkQaInput(chunk=text, question=question, generate_highlights=True)

# Define a LuminousControlModel and instantiate a SingleChunkQa task
model = LuminousControlModel(name="luminous-base-control")
single_chunk_qa = SingleChunkQa(model=model)

output = single_chunk_qa.run(input, NoOpTracer())

Pretty(output.answer)

Nice, we extracted some advantages!

If you want to investigate based on which part of the input text the answer was produced, you can use the `highlights` property of the `SingleChunkQaOutput`.
Under the hood, is uses the explainability feature of the Aleph Alpha inference stack. Each highlight in the `highlights` list contains the start and end courser position of the relevant text section and a score indicating its degree of relevance.

In [ ]:
output.highlights

In [ ]:
print(
    f"Highlight 1 (Score {round(output.highlights[0].score,2)}): {text[output.highlights[0].start:output.highlights[0].end]}"
)
print(
    f"Highlight 2 (Score {round(output.highlights[1].score,2)}): {text[output.highlights[1].start:output.highlights[1].end]}"
)

What if we ask a question that cannot be answered on the basis of the text?

In [ ]:
question = "Who plays Gandalf in LOTR?"
input = SingleChunkQaInput(chunk=text, question=question)
output = single_chunk_qa.run(input, NoOpTracer())

output

As you can see, our QA-task returns *None*, because there is no answer to our question within the source document.

Thus, we prevented a potential hallucination.

## Language detection

You can also ask questions about documents in languages other than English.
Our models support 5 European languages:
- English - 'en'
- German - 'de'
- Spanish - 'es'
- French - 'fr'
- Italian - 'it'
  
We provide you with some tools making it easier to detect the language in the document.

In [ ]:
document_with_unknown_language = """Rom begann ab dem 5. Jahrhundert v. Chr. mit einer immer rascheren Expansion in Mittelitalien (Eroberung von Veji 396 v. Chr.), musste dabei aber auch schwere Rückschläge verkraften. Der „Galliersturm“ unter Brennus hinterließ psychologisch tiefe Spuren, wobei die Schlacht an der Allia am 18. Juli (wahrscheinlich) 387 v. Chr. als „dies ater“ („schwarzer Tag“) in die Geschichte Roms einging. Es folgten die Samnitenkriege (343–341 v. Chr.; 326–304 v. Chr.; 298–290 v. Chr.) und der Latinerkrieg (um 340–338 v. Chr.). Rom schuf schließlich ein weitverzweigtes Bündnisgeflecht. So wurden an strategisch wichtigen Orten Kolonien angelegt und Bündnisse mit mehreren italischen Stämmen geschlossen, die jedoch nicht das römische Bürgerrecht erhielten.

Aus dieser Zeit seiner Geschichte ging Rom als straffes Staatswesen mit schlagkräftiger Armee und starkem Drang zur Ausdehnung hervor. Damit waren die Grundlagen für seinen weiteren Aufstieg geschaffen. Konkurrierende Mächte stellten auf der Italischen Halbinsel die Stadtstaaten der Etrusker nördlich von Rom, die Kelten in der Po-Ebene und die griechischen Kolonien in Süditalien dar.

Im 3. Jahrhundert v. Chr. setzte sich Rom gegen die Samniten und andere italische Stämme durch. Nach und nach fiel die gesamte Halbinsel an Rom (außer Oberitalien, welches erst später annektiert wurde). Im Süden verleibte sich die Republik um 275 v. Chr. die dortigen griechischen Stadtstaaten ein, nachdem es während des Pyrrhischen Krieges gelungen war, den hellenistischen Hegemon Pyrrhos I. von Epiros abzuwehren. Mit dieser Expansion kam Rom allerdings in Konflikt mit der bisher Rom freundlich gesinnten Handelsrepublik Karthago (im heutigen Tunesien), was zu den Punischen Kriegen führte."""

lang_detection_input = DetectLanguageInput(
    text=document_with_unknown_language,
    possible_languages=[
        Language(language) for language in ["en", "de", "es", "fr", "it"]
    ],
)
language = DetectLanguage().run(lang_detection_input, NoOpTracer())
language

Ah, it seems that this document is written in German!
Let's generate a German answer.

In [ ]:
question = "Wie viele Samnitenkriege gab es & wann fanden sie statt?"

input = SingleChunkQaInput(
    chunk=document_with_unknown_language,
    question=question,
    language=language.best_fit,
)
output = single_chunk_qa.run(input, NoOpTracer())

print(output.answer)

## Multi-chunk QA

Some times you might have multiple texts you want to provide as context for your question. In this case the `MultipleChunkQa`-task might be the better option. The workflow of this task consists of the following steps:
1. The tasks takes multiple text chunks and a question as input.
2. It runs the model for each chunk generating an individual answer per chunk.
3. It generates a final answer based on the combination of the intermediate answers.

Note, that for the  `MultipleChunkQa` the combined length of all input chunks is **not** limited by the context window of the model. Thus, `MultipleChunkQa` provides one option to deal with long input texts by splitting them into multiple chunks. However, below in the section [Long context QA](#long-context-qa) we will present a more sophisticated approache on how to handle QA-tasks for long input texts. 

Now let's have a look at an example where two chunks lead to different parts of the final answer.

This time, let's also use a proper debug log, so that we can see what happens under the hood!

In [ ]:
chunks = [
    'Around 1997, Goldenvoice was struggling to book concerts against larger companies, and they were unable to offer guarantees as high as their competitors, such as SFX Entertainment. Tollett said, "We were getting our ass kicked financially. We were losing a lot of bands. And we couldn\'t compete with the money." As a result, the idea of a music festival was conceived, and Tollett began to brainstorm ideas for one with multiple venues. His intent was to book trendy artists who were not necessarily chart successes: "Maybe if you put a bunch of them together, that might be a magnet for a lot of people." While attending the 1997 Glastonbury Festival, Tollett handed out pamphlets to artists and talent managers that featured pictures of the Empire Polo Club and pitched a possible festival there. In contrast to the frequently muddy conditions at Glastonbury caused by rain, he recalled, "We had this pamphlet... showing sunny Coachella. Everyone was laughing."',
    "Rock am Ring wurde erstmals 1985 veranstaltet und war ursprünglich als ein einmaliges Ereignis geplant. Aufgrund des großen Erfolges mit 75.000 Zuschauern entschloss man sich jedoch, diese Veranstaltung jedes Jahr stattfinden zu lassen. Der Einbruch der Zuschauerzahlen 1988 hatte eine zweijährige Pause zur Folge. 1991 startete das größte deutsche Rockfestival mit einem überarbeiteten Konzept erneut. Ein neues Hauptaugenmerk wurde darauf gelegt, dem Publikum mehr Newcomer vorzustellen. So traten unter anderem die zu diesem Zeitpunkt eher unbekannten INXS oder Alanis Morissette bei Rock am Ring vor großem Publikum auf.",
]

question = "What festival is the text about?"
input = MultipleChunkQaInput(chunks=chunks, question=question)

multi_chunk_qa = MultipleChunkQa(merge_answers_model=model)
tracer = InMemoryTracer()
output = multi_chunk_qa.run(input, tracer)

In [ ]:
print(output.answer)

Notice how information from each chunk made it into the final combined answer; despite the texts being written in different languages.

Furthermore, it is also possible to examine the inner workings of the QA mechanism by looking at the `tracer`. There, we track the inputs, outputs, and internal states used to calculate the mentioned outputs. You can also take a look at the parameters of the request sent to the API.

In [ ]:
tracer

From the trace we can see that the `MultipleChunkQa`-task runs the the `SingleChunkQa` twice, once for each chunk and then combines both answeres in a final `Complete`. 

## Long context QA

Sometimes you want to execute a QA request against a document that is really, really long.
In such cases, the document will not fit into the context window of the model (e.g., 2048 tokens for Luminous) and not all parts of it will be relevant for the question at hand.
We will first need to split the document into chunks and use semantic search to find the most relevant chunks (i.e., those most similar to the question).
Then, we use these chunks to answer the question using the `MultipleChunkQa` functionality.

In [ ]:
# https://en.wikipedia.org/wiki/Robert_Moses
long_text = """Robert Moses''' (December 18, 1888 – July 29, 1981) was an American [[urban planner]] and public official who worked in the [[New York metropolitan area]] during the early to mid 20th century. Despite never being elected to any office, Moses is regarded as one of the most powerful and influential individuals in the history of New York City and New York State. The grand scale of his infrastructural projects and his philosophy of urban development influenced a generation of engineers, architects, and urban planners across the United States.<ref name=":0" />

Moses held various positions throughout his more than forty-year long career. He at times held up to 12 titles simultaneously, including [[New York City Parks Commissioner]] and chairman of the [[Long Island State Park Commission]].<ref>{{Cite web|url=https://www.pbs.org/wnet/need-to-know/environment/the-legacy-of-robert-moses/16018/|title=The legacy of Robert Moses|last=Sarachan|first=Sydney|date=January 17, 2013|website=Need to Know {{!}} PBS|language=en-US|access-date=December 3, 2019}}</ref> Having worked closely with New York governor [[Al Smith]] early in his career, Moses became expert in writing laws and navigating and manipulating the inner workings of state government. He created and led numerous semi-autonomous [[Public authority|public authorities]], through which he controlled millions of dollars in revenue and directly issued [[Bond (finance)|bonds]] to fund new ventures with little outside input or oversight.

Moses's projects transformed the New York area and revolutionized the way cities in the U.S. were designed and built. As Long Island State Park Commissioner, Moses oversaw the construction of [[Jones Beach State Park]], the most visited public beach in the United States,<ref name="Jones Beach">{{cite news |url=http://www.longislandexchange.com/jones-beach.html |website=Long Island Exchange |title=Jones Beach |access-date=November 21, 2012 |archive-url=https://web.archive.org/web/20130121130008/http://www.longislandexchange.com/jones-beach.html |archive-date=January 21, 2013 |url-status=dead }}</ref> and was the primary architect of the [[Parkways in New York|New York State Parkway System]]. As head of the [[MTA Bridges and Tunnels|Triborough Bridge Authority]], Moses had near-complete control over bridges and tunnels in New York City as well as the tolls collected from them, and built, among others, the [[Robert F. Kennedy Bridge|Triborough Bridge]], the [[Brooklyn–Battery Tunnel]], and the [[Throgs Neck Bridge]], as well as several major highways. These roadways and bridges, alongside [[urban renewal]] efforts that saw the destruction of huge swaths of tenement housing and their replacement with large [[New York City Housing Authority|public housing projects]], transformed the physical fabric of New York and inspired other cities to undertake similar development endeavors.

Moses's reputation declined following the publication of [[Robert Caro]]'s [[Pulitzer Prize]]-winning biography ''[[The Power Broker]]'' (1974), which cast doubt on the purported benefits of many of Moses's projects and further cast Moses as racist. In large part because of ''The Power Broker'', Moses is today considered a controversial figure in the history of New York City.

==Early life and career==
Moses was born in [[New Haven, Connecticut]], on December 18, 1888, to [[German Jewish]] parents, Bella (Silverman) and Emanuel Moses.<ref>{{cite news | url=https://www.nytimes.com/learning/general/onthisday/bday/1218.html | title=Robert Moses, Master Builder, is Dead at 92| newspaper=The New York Times |archive-url=https://web.archive.org/web/20160305003155/https://www.nytimes.com/learning/general/onthisday/bday/1218.html |archive-date=March 5, 2016 |url-status=dead}}</ref>{{sfn|Caro|1974|p=25}} He spent the first nine years of his life living at 83 Dwight Street in New Haven, two blocks from [[Yale University]]. In 1897, the Moses family moved to New York City,{{sfn|Caro|1974|pp=29}} where they lived on East 46th Street off Fifth Avenue.<ref>{{cite web |url=http://www.newsday.com/community/guide/lihistory/ny-history-hs722a,0,7092161.story |title=The Master Builder |access-date=April 4, 2007 |last=DeWan |first=George |year=2007 |website=Long Island History |publisher=Newsday |archive-url=https://web.archive.org/web/20061211045554/http://www.newsday.com/community/guide/lihistory/ny-history-hs722a%2C0%2C7092161.story |archive-date=December 11, 2006 |url-status=dead  }}</ref> Moses's father was a successful department store owner and [[real estate]] speculator in New Haven. In order for the family to move to New York City, he sold his real estate holdings and store, then retired.{{sfn|Caro|1974|pp=29}} Moses's mother was active in the [[settlement movement]], with her own love of building. Robert Moses and his brother Paul attended several schools for their elementary and [[secondary education]], including the [[Ethical Culture Fieldston School|Ethical Culture School]], the [[Dwight School]] and the [[Mohegan Lake, New York#Historic places|Mohegan Lake School]], a military academy near [[Peekskill, New York|Peekskill]].{{sfn|Caro|1974|pp=35}}

After graduating from [[Yale College]] (B.A., 1909) and [[Wadham College]], [[Oxford University|Oxford]] (B.A., Jurisprudence, 1911; M.A., 1913), and earning a Ph.D. in [[political science]] from [[Columbia University]] in 1914, Moses became attracted to New York City reform politics.<ref>{{Cite web|url=http://c250.columbia.edu/c250_celebrates/remarkable_columbians/robert_moses.html|title = Robert Moses}}</ref> A committed [[idealism|idealist]], he developed several plans to rid New York of [[Patronage#Politics|patronage hiring]] practices, including being the lead author of a 1919 proposal to reorganize the New York state government. None went very far, but Moses, due to his intelligence, caught the notice of [[Belle Moskowitz]], a friend and trusted advisor to Governor [[Al Smith]].{{sfn|Caro|1974}}  When the state [[Secretary of State of New York|Secretary of State's]] position became appointive rather than elective, Smith named Moses. He served from 1927 to 1929.<ref>{{cite news |date=December 19, 1928 |title=Moses Resigns State Position |url=http://cdsun.library.cornell.edu/cgi-bin/cornell?a=d&d=CDS19281219.2.63.7# |newspaper=Cornell Daily Sun |location=Ithaca, NY |page=8}}</ref>

Moses rose to power with Smith, who was elected as governor in 1918, and then again in 1922. With Smith's support, Moses set in motion a sweeping consolidation of the New York State government. During that period Moses began his first foray into large-scale public work initiatives, while drawing on Smith's political power to enact legislation. This helped create the new [[Long Island State Park Commission]] and the State Council of Parks.<ref>{{cite web|last=Gutfreund|first=Owen|title=Moses, Robert|url=http://www.anb.org/articles/07/07-00375.html|publisher=Anb.org|access-date=December 24, 2014}}</ref> In 1924, Governor Smith appointed Moses chairman of the State Council of Parks and president of the Long Island State Park Commission.<ref>{{Cite book|title=Encyclopedia of the City|url=https://archive.org/details/encyclopediacity00cave|url-access=limited|last=Caves|first=R. W.|publisher=Routledge|year=2004|isbn=978-0-415-25225-6|pages=[https://archive.org/details/encyclopediacity00cave/page/n512 472]}}</ref> This centralization allowed Smith to run a government later used as a model for Franklin D. Roosevelt's [[New Deal]] federal government.{{or|date=October 2022}} Moses also received numerous commissions that he carried out efficiently, such as the development of [[Jones Beach State Park]].{{cn|date=October 2022}} Displaying a strong command of [[law]] as well as matters of [[engineering]], Moses became known for his skill in drafting legislation, and was called "the best bill drafter in [[Albany, New York|Albany]]".<ref name=":0">{{cite news |title=Annals of Power |first=Robert A. |last=Caro |author-link=Robert Caro |url=http://archives.newyorker.com/?i=1974-07-22#folio=032 |magazine=[[The New Yorker]] |date=July 22, 1974 |access-date=September 1, 2011}}</ref> At a time when the public was accustomed to [[Tammany Hall]] corruption and incompetence, Moses was seen as a savior of government.{{sfn|Caro|1974}}

Shortly after [[President of the United States|President]] [[Franklin Delano Roosevelt|Franklin D. Roosevelt's]] [[First inauguration of Franklin D. Roosevelt|inauguration]] in 1933, the [[United States federal government|federal government]] found itself with millions of [[New Deal]] dollars to spend, yet states and cities had few projects ready. Moses was one of the few local officials who had projects [[shovel ready]]. For that reason, New York City was able to obtain significant [[Works Progress Administration]] (WPA), [[Civilian Conservation Corps]] (CCC), and other Depression-era funding. One of his most influential and longest-lasting positions was that of Parks Commissioner of New York City, a role he served from January 18, 1934, to May 23, 1960.<ref>{{Cite web|url=https://www.nycgovparks.org/about/history/commissioners|title=New York City Parks Commissioners : NYC Parks|website=www.nycgovparks.org|language=en|access-date=March 29, 2018}}</ref>

==Offices held==
The many offices and professional titles that Moses held gave him unusually broad power to shape urban development in the New York metropolitan region. These include, according to the New York Preservation Archive Project:<ref>{{Cite web|url=http://www.nypap.org/preservation-history/robert-moses/|title=Robert Moses {{!}}|website=www.nypap.org|language=en-US|access-date=March 29, 2018}}</ref>
*[[Long Island State Park Commission]] (President, 1924–1963)
* New York State Council of Parks (Chairman, 1924–1963)
*[[Secretary of State of New York|New York Secretary of State]] (1927–1928)
* Bethpage State Park Authority (President, 1933–1963)
* Emergency Public Works Commission (Chairman, 1933–1934)
* Jones Beach Parkway Authority (President, 1933–1963)
*[[New York City Department of Parks and Recreation|New York City Department of Parks]] (Commissioner, 1934–1960)
* [[Triborough Bridge]] and Tunnel Authority (Chairman, 1934–1968)
* New York City Planning Commission (Commissioner, 1942–1960)
* New York State Power Authority (Chairman, 1954–1962)
* [[1964 New York World's Fair|New York's World Fair]] (President, 1960–1966)
* Office of the Governor of New York (Special Advisor on Housing, 1974–1975)

==Influence==
During the 1920s, Moses sparred with [[Franklin D. Roosevelt]], then head of the Taconic State Park Commission, who favored the prompt construction of a [[parkway]] through the [[Hudson Valley]]. Moses succeeded in diverting funds to his Long Island parkway projects (the [[Northern State Parkway]], the [[Southern State Parkway]] and the [[Wantagh State Parkway]]), although the [[Taconic State Parkway]] was later completed as well.<ref>{{cite web|url=http://www.nycroads.com/roads/taconic/ |title=Taconic State Parkway |website=NYCRoads.com |access-date=May 25, 2006}}</ref> Moses helped build Long Island's [[Meadowbrook State Parkway]]. It was the first fully divided limited access highway in the world.<ref name="Leonard 1991 339">{{cite book|last=Leonard|first=Wallock|title=The Myth of The Master Builder|year=1991|publisher=Journal of Urban History|page=339}}</ref>

Moses was a highly influential figure in the initiation of many of the reforms that restructured New York state's government during the 1920s. A 'Reconstruction Commission' headed by Moses produced a highly influential report that provided recommendations that would largely be adopted, including the consolidation of 187 existing agencies under 18 departments, a new executive budget system, and the four-year term limit for the governorship.{{sfn|Caro|1974|pp=106, 260}}"""

In [ ]:
question = "What is the name of the book about Robert Moses?"
input = LongContextQaInput(text=long_text, question=question)

long_context_qa = LongContextQa(model=model)
tracer = InMemoryTracer()
output = long_context_qa.run(input, tracer=tracer)

In [ ]:
print(output.answer)

Let's have a look at the source chunk for this answer!

In [ ]:
print(
    "Number of subanswers that contributed to the final answer:", len(output.subanswers)
)
print("Subanswer:", output.subanswers[0].answer)
print("Chunk:", output.subanswers[0].chunk)